### Model Evaluation with Batch Normalization

In [5]:
import numpy as np
import tensorflow as tf
import random as python_random
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def get_preprocessed_data(images, labels):
    
    images = np.array(images/255.0, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    labels = labels.squeeze()
    
    return images, labels

def get_preprocessed_ohe(images, labels):
    
    images, labels = get_preprocessed_data(images, labels)
    oh_labels = to_categorical(labels)
    
    return images, oh_labels

In [6]:
(train_images, train_labels), (test_images, test_labels) = \
    cifar10.load_data()

print('train datasets shape: ', train_images.shape, train_labels.shape)
print('test datasets shape: ', test_images.shape, test_labels.shape)

train datasets shape:  (50000, 32, 32, 3) (50000, 1)
test datasets shape:  (10000, 32, 32, 3) (10000, 1)


In [7]:
train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
print('train datasets shape: ', train_images.shape, train_oh_labels.shape)

train datasets shape:  (50000, 32, 32, 3) (50000, 10)


In [8]:
test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
print('test datasets shape: ', test_images.shape, test_oh_labels.shape)

test datasets shape:  (10000, 32, 32, 3) (10000, 10)


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, Activation, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

IMAGE_SIZE=32

input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(input_tensor)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
x = Activation('relu')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Flatten(name='flatten')(x)
x = Dropout(rate=0.5)(x)
x = Dense(300, activation='relu', name='fc1')(x)
x = Dropout(rate=0.3)(x)
output = Dense(10, activation='softmax', name='output')(x)

model = Model(inputs=input_tensor, outputs=output)

model.summary()

Metal device set to: Apple M1


2021-08-16 20:19:21.023085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-16 20:19:21.024049: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0     

In [11]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=train_images, y=train_oh_labels, batch_size=64, epochs=30, validation_split=0.15)

2021-08-16 20:19:55.455405: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-16 20:19:55.459459: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/30


2021-08-16 20:19:55.734995: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


665/665 [==============================] - ETA: 0s - loss: 1.5529 - accuracy: 0.4384

2021-08-16 20:20:21.184456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


665/665 [==============================] - 27s 36ms/step - loss: 1.5529 - accuracy: 0.4384 - val_loss: 1.3151 - val_accuracy: 0.5095
Epoch 2/30
665/665 [==============================] - 22s 34ms/step - loss: 1.1012 - accuracy: 0.6083 - val_loss: 1.0479 - val_accuracy: 0.6259
Epoch 3/30
665/665 [==============================] - 23s 35ms/step - loss: 0.9040 - accuracy: 0.6824 - val_loss: 1.0920 - val_accuracy: 0.6077
Epoch 4/30
665/665 [==============================] - 24s 36ms/step - loss: 0.7935 - accuracy: 0.7246 - val_loss: 0.9155 - val_accuracy: 0.6801
Epoch 5/30
665/665 [==============================] - 22s 33ms/step - loss: 0.7072 - accuracy: 0.7546 - val_loss: 0.7603 - val_accuracy: 0.7368
Epoch 6/30
665/665 [==============================] - 22s 32ms/step - loss: 0.6352 - accuracy: 0.7801 - val_loss: 0.8016 - val_accuracy: 0.7321
Epoch 7/30
665/665 [==============================] - 22s 32ms/step - loss: 0.5841 - accuracy: 0.7993 - val_loss: 0.6631 - val_accuracy: 0.7759
Epo

In [12]:
model.evaluate(test_images, test_oh_labels)

313/313 [==============================] - 3s 10ms/step - loss: 0.6724 - accuracy: 0.8315


[0.6723729968070984, 0.8315000534057617]

### Applying he normal initialization as the above model

In [13]:
input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal')(input_tensor)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Flatten(name='flatten')(x)
x = Dropout(rate=0.5)(x)
x = Dense(300, activation='relu', name='fc1')(x)
x = Dropout(rate=0.3)(x)
output = Dense(10, activation='softmax', name='output')(x)

model = Model(inputs=input_tensor, outputs=output)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0   

In [14]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=train_images, y=train_oh_labels, batch_size=64, epochs=30, validation_split=0.15)

Epoch 1/30
  1/665 [..............................] - ETA: 4:21 - loss: 3.9104 - accuracy: 0.1250

2021-08-16 20:37:47.618973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


664/665 [============================>.] - ETA: 0s - loss: 1.6178 - accuracy: 0.4173

2021-08-16 20:38:09.528761: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


665/665 [==============================] - 23s 35ms/step - loss: 1.6179 - accuracy: 0.4173 - val_loss: 1.5891 - val_accuracy: 0.4444
Epoch 2/30
665/665 [==============================] - 24s 36ms/step - loss: 1.1757 - accuracy: 0.5790 - val_loss: 1.5767 - val_accuracy: 0.4759
Epoch 3/30
665/665 [==============================] - 24s 36ms/step - loss: 0.9674 - accuracy: 0.6568 - val_loss: 1.1596 - val_accuracy: 0.5989
Epoch 4/30
665/665 [==============================] - 24s 36ms/step - loss: 0.8370 - accuracy: 0.7072 - val_loss: 0.9777 - val_accuracy: 0.6688
Epoch 5/30
665/665 [==============================] - 24s 36ms/step - loss: 0.7480 - accuracy: 0.7381 - val_loss: 1.2911 - val_accuracy: 0.6257
Epoch 6/30
665/665 [==============================] - 24s 36ms/step - loss: 0.6868 - accuracy: 0.7625 - val_loss: 0.7345 - val_accuracy: 0.7451
Epoch 7/30
665/665 [==============================] - 24s 36ms/step - loss: 0.6284 - accuracy: 0.7803 - val_loss: 0.7900 - val_accuracy: 0.7361
Epo

In [15]:
model.evaluate(test_images, test_oh_labels, batch_size=64)

157/157 [==============================] - 2s 11ms/step - loss: 0.8168 - accuracy: 0.8084


[0.8168178200721741, 0.80840003490448]